In [1]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [34]:
!pip install rasterio pandas numpy

In [35]:
import rasterio as rio
from rasterio.windows import Window
import pandas as pd
import numpy as np

In [36]:
# img = rio.open(r'/content/data/Images/sentinel2_2017-07-31.tiff')

with rio.open("/content/data/Images/sentinel2_2017-07-31.tiff") as src:
  data = src.read(1)
  meta = src.meta

In [37]:
array = img.read()
n_bands = array.shape[0]

In [38]:
height = img.shape[0]
width = img.shape[1]
cols, rows = np.meshgrid(np.arange(width), np.arange(height))
xs, ys = rio.transform.xy(img.transform, rows, cols)
xcoords = np.array(xs)
ycoords = np.array(ys)

In [39]:
array = np.concatenate((array, xcoords[None,:,:], ycoords[None,:,:]))

In [40]:
df = pd.DataFrame(array.reshape([n_bands+2,-1]).T, columns=[f"band_{i+1}" for i in range(n_bands)]+['x','y'])

In [41]:
with rio.open("/content/data/Images/sentinel2_2017-07-31.tiff") as src:
  data = src.read(1)
  meta = src.meta

In [42]:
width, height = data.shape[1], data.shape[0]
x_coords = [i * meta['transform'][0] for i in range(width)]
y_coords = [height - j * meta['transform'][4] for j in range(height)]

x_coords = [x if x is not None else np.nan for x in x_coords]
y_coords = [y if y is not None else np.nan for y in y_coords]

In [43]:
try:
  df = pd.DataFrame({'X': x_coords, 'Y': y_coords})

  for band in range(data.shape[0]):
    df[f'Band_{band+1}'] = data[band].flatten()

except ValueError as e:
  print("Error:", e)

  print(df)

Error: All arrays must be of the same length
         band_1  band_2  band_3  band_4  band_5  band_6  band_7  band_8  \
0        0.0428  0.0403  0.0618  0.0685  0.1063  0.1593  0.1702  0.1747   
1        0.0428  0.0416  0.0643  0.0715  0.1063  0.1593  0.1702  0.1740   
2        0.0392  0.0441  0.0629  0.0721  0.1084  0.1618  0.1799  0.1830   
3        0.0392  0.0384  0.0595  0.0644  0.1084  0.1618  0.1799  0.1915   
4        0.0392  0.0442  0.0607  0.0697  0.1227  0.1602  0.1797  0.1863   
...         ...     ...     ...     ...     ...     ...     ...     ...   
1233351  0.0375  0.0414  0.0691  0.0805  0.1570  0.2139  0.2387  0.2376   
1233352  0.0375  0.0643  0.0926  0.1185  0.1570  0.2139  0.2387  0.2528   
1233353  0.0375  0.0469  0.0795  0.0937  0.1537  0.1933  0.2070  0.2227   
1233354  0.0375  0.0425  0.0685  0.0807  0.1537  0.1933  0.2070  0.2133   
1233355  0.0375  0.0484  0.0788  0.0947  0.1308  0.1812  0.1952  0.2181   

         band_9  band_10  band_11  band_12  band_13   

In [21]:
print(f"Length of x_coords: {len(x_coords)}")
print(f"Length of y_coords: {len(y_coords)}")
print(data.shape)

Length of x_coords: 1226
Length of y_coords: 1006
(1006, 1226)
